Import necessary module

In [ ]:
#prepare inputs for InceptionV3 (299x299x3) ---
from tensorflow.keras.applications.inception_v3 import preprocess_input
import tensorflow as tf

def to_inception_input(x):
    x = tf.convert_to_tensor(x)
    if x.shape.rank == 3:            # single image -> add batch
        x = tf.expand_dims(x, 0)
    if x.shape[-1] == 1:
        x = tf.repeat(x, repeats=3, axis=-1)
    x = tf.image.resize(x, (299, 299))
    x = tf.cast(x, tf.float32)
    # InceptionV3 preprocess scales to [-1, 1]
    return preprocess_input(x)

# If you already have MNIST in x_subset (N,28,28) or (N,28,28,1), build x_subset_pp:
# x_subset_pp = to_inception_input(x_subset)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, LocallyLinearEmbedding
import umap.umap_ as umap

**Load and Preprocessing MNIST**

In [ ]:
(trainX, trainY), _ = mnist.load_data()
trainX = trainX.astype("float32") / 255.0

def preprocess_images_batch(images, batch_size=500):
    resized = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i+batch_size]
        batch = tf.expand_dims(batch, -1)           # (28,28) -> (28,28,1)
        batch = tf.image.resize(batch, (299, 299)) # resize to 224x224
        batch = tf.image.grayscale_to_rgb(batch)   # (299, 299,1) -> (299, 299, 3)
        resized.append(batch)
    return tf.concat(resized, axis=0).numpy()

x_subset = preprocess_images_batch(trainX[:2000])
y_subset = trainY[:2000]
y_subset_cat = to_categorical(y_subset, num_classes=10)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3), pooling='avg')
base_model.trainable = False

feature_extractor_before = Model(inputs=base_model.input, outputs=base_model.output)
features_before = feature_extractor_before.predict(x_subset, batch_size=64, verbose=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 501ms/step


Fine Tuning

In [ ]:
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = layers.Dense(128, activation='relu', name="features")(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune on subset
x_train_small = preprocess_images_batch(trainX[:5000])
y_train_small = to_categorical(trainY[:5000], num_classes=10)

model.fit(x_train_small, y_train_small, epochs=10, batch_size=64, validation_split=0.1)

NameError: name 'tf' is not defined

**Extract Features After Fine-tuning from intermediate Dense layer**

In [ ]:
feature_extractor_after = Model(inputs=model.input, outputs=model.get_layer("features").output)
features_after = feature_extractor_after.predict(x_subset_pp, batch_size=64, verbose=1)

**VIsualization and Plots(PCA,t-SNE,UMAP)**

In [ ]:
def plot_2d(X_before, y_before, X_after, y_after, title_before, title_after):
    plt.figure(figsize=(16, 6))

    plt.subplot(1,2,1)
    scatter = plt.scatter(X_before[:, 0], X_before[:, 1], c=y_before, cmap='tab10', s=10, alpha=0.8)
    plt.legend(*scatter.legend_elements(), title="Digits", loc='best')
    plt.title(title_before)

    plt.subplot(1,2,2)
    scatter = plt.scatter(X_after[:, 0], X_after[:, 1], c=y_after, cmap='tab10', s=10, alpha=0.8)
    plt.legend(*scatter.legend_elements(), title="Digits", loc='best')
    plt.title(title_after)

    plt.grid(True)
    plt.show()

# PCA
pca = PCA(n_components=2)
pca_before = pca.fit_transform(features_before)
pca_after = pca.fit_transform(features_after)
plot_2d(pca_before, y_subset, pca_after, y_subset, "PCA - Before Transfer Learning", "PCA - After Transfer Learning")


# t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000)
tsne_before = tsne.fit_transform(features_before)
tsne_after = tsne.fit_transform(features_after)
plot_2d(tsne_before, y_subset, tsne_after, y_subset, "t-SNE - Before Transfer Learning", "t-SNE - After Transfer Learning")

# UMAP
umap_model = umap.UMAP(n_components=2)
umap_before = umap_model.fit_transform(features_before)
umap_after = umap_model.fit_transform(features_after)
plot_2d(umap_before, y_subset, umap_after, y_subset, "UMAP - Before Transfer Learning", "UMAP - After Transfer Learning")